In [8]:
# Imports:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import feature_extraction, linear_model, model_selection

In [ ]:
# Library Imports:
import numpy as np
import pandas as pd
import statistics as st
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import regex
import gc
import re
import string
import operator
from collections import defaultdict
import sys
import tokenization
from wordcloud import STOPWORDS
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

#  Metrics Import:
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [2]:
# Reading in Datasets:
df_train_cleanedMislabelsDuplicates = pd.read_csv("../Data/df_train_cleanedMislabelsDuplicates.csv", encoding='utf-8')
df_train_cleanedNoMislabelsDuplicates = pd.read_csv("../Data/df_train_cleanedNoMislabelsDuplicates.csv", encoding='utf-8')
df_train_cleanedMislabelsNoDuplicates = pd.read_csv("../Data/df_train_cleanedMislabelsNoDuplicates.csv", encoding='utf-8')
df_train_cleanedNoMislabelsNoDuplicates = pd.read_csv("../Data/df_train_cleanedNoMislabelsNoDuplicates.csv", encoding='utf-8')
df_test_cleaned = pd.read_csv("../Data/df_test_cleaned.csv", encoding='utf-8')

#### Building Vectors:
We use sckit-learn's CountVectorizer to count the words in each tweet and turn them into data our machine learning model can process

In [16]:
count_vectorizer = feature_extraction.text.CountVectorizer()

train_vectors = count_vectorizer.fit_transform(df_train_cleanedNoMislabelsNoDuplicates['text'])

#For test_vectors, we only use .transform() function b/c we need that the train and test vectors use the same set of tokens
test_vectors = count_vectorizer.transform(df_test_cleaned["text"])

#### Building Model:
We use the words contained in each tweet as the predictor variable for real/fake disaster tweet (1/0). We assume a linear model/connection in this tutorial

In [17]:
# Since the vectors are big, we want to push the model's weights toward 0 without completely discounting different words
# Thus, we use Ridge Regression
clf = linear_model.RidgeClassifier()
# cv = 3 means we are using Three-Fold Cross Validation
scores = model_selection.cross_val_score(clf, train_vectors, df_train_cleanedNoMislabelsNoDuplicates['target_relabeled'], cv = 3, scoring = "f1")
print("Scores: ", scores)

Scores:  [0.5767098  0.53946621 0.5949214 ]


Ways to improve this score: Do TFIDF, LSA, LSTM/RNNs, etc

#### Predictions and Submission File:

In [10]:
clf.fit(train_vectors, train_df['target'])
tutorial_submission = pd.read_csv("../Data/sample_submission.csv")
tutorial_submission['target'] = clf.predict(test_vectors)
tutorial_submission.to_csv("../Result Files/submission.csv", index = False)